In [1]:
import os,sys,inspect
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir) 
from GGA import GGA
import numpy as np

In [2]:
class Testing1():
    def __init__(self,data,model,verbose=0):
        self.data = data
        self.model = model
        self.verbose = verbose
    
    def testRow(self,row_prev,row,row_pred):
        totalMatchDynamic = 0
        totalMatchStatic = 0
        totalStatic = 0
        totalDynamic = 0
        for cell0,cell1,cell2 in zip(row_prev,row,row_pred):
            totalMatchDynamic += (cell0 != cell1 and cell1 == cell2)
            totalMatchStatic += (cell0 == cell1 and cell1 == cell2)
            totalStatic += (cell0 == cell1)
            totalDynamic += (cell0 != cell1)
        totalStatic = totalStatic if totalStatic > 0 else 0
        totalDynamic = totalDynamic if totalDynamic > 0 else 0
        return [totalStatic,totalDynamic,totalMatchStatic,totalMatchDynamic]
            
    def run(self):
        res = []
        backend = "threading"
        with Parallel(n_jobs=4,backend=backend) as parallel:
            if self.verbose >= 1: print("Testing:")
            for i in range(1,len(data)):
                if self.verbose >= 1: print(f"state {i}/{len(data)}")
                prediction = next(model.run(data[i-1],1))
                rowsTotal = parallel(delayed(self.testRow)(row0,row1,row2) for row0,row1,row2 in zip(data[i-1],data[i],prediction[1]))
                summation = np.sum(rowsTotal,axis=0)
                if self.verbose >= 1: 
                    print(f"Static : {summation[2]}/{summation[0]}")
                    print(f"Dynamic : {summation[3]}/{summation[1]}")
                totalStatic = summation[2]/summation[0] if summation[0] > 0 else 0
                totalDynamic = summation[3]/summation[1] if summation[1] > 0 else 0
                res.append([totalStatic,totalDynamic])
        self.result = np.array(res)
    
    def plot(self):
        plt.plot(range(len(self.result)),self.result[:,0])
        plt.plot(range(len(self.result)),self.result[:,1])
        return plt

import sys
del sys.modules["ExperimentBase"]

In [3]:
data = np.array([[0,2,0,0,2],[0,1,0,1,2],[1,0,1,1,2],[1,0,0,1,2],
        [1,0,1,0,1],[0,0,0,1,1],[1,1,1,1,1],[0,0,0,0,1],[1,0,0,0,1],[1,1,1,0,1]])

In [4]:
model = GGA(data,4,path="saves/test",verbose=2,noChilds=10,noChromosomes=100)

start setup
0.0
(10, 3)
0.0
0.0009996891021728516
0.0009996891021728516
2 goals
end setup


In [5]:
model.summarized

{'total': {2: 4, 1: 6, 'total': 10},
 0: {0: {2: 2, 'total': 4, 1: 2}, 1: {2: 2, 'total': 6, 1: 4}},
 1: {2: {2: 1, 'total': 1},
  1: {2: 1, 'total': 3, 1: 2},
  0: {2: 2, 'total': 6, 1: 4}},
 2: {0: {2: 3, 'total': 6, 1: 3}, 1: {2: 1, 'total': 4, 1: 3}},
 3: {0: {2: 1, 'total': 5, 1: 4}, 1: {2: 3, 'total': 5, 1: 2}}}

In [6]:
model.run()

config saved at saves/test/29_01_20_18_16_05/config.json
iteration 1
goal = 2goal = 1

goal 2 elapsed time: 0.08399844169616699
goal 1 elapsed time: 0.09399914741516113
{'generation': 1, 'elapsedTime': 0.10399985313415527, 'fitness': {2: [0.004744224422442184, -0.041666666666666706, 0.2777777777777777], 1: [0.016570407040704123, -0.1666666666666666, 0.3055555555555555]}}
iteration 2
goal = 2goal = 1

goal 1 elapsed time: 0.01599884033203125goal 2 elapsed time: 0.019998788833618164

{'generation': 2, 'elapsedTime': 0.10299944877624512, 'fitness': {2: [0.02138338833883382, 0.11111111111111105, 0.2777777777777777], 1: [0.02963421342134219, -0.0416666666666666, 0.3055555555555555]}}
iteration 3
goal = 2
goal = 1
goal 2 elapsed time: 0.013001680374145508
goal 1 elapsed time: 0.014997005462646484
{'generation': 3, 'elapsedTime': 0.10299897193908691, 'fitness': {2: [0.04036028602860281, 0.18749999999999994, 0.2777777777777778], 1: [0.04166666666666673, 0.18749999999999994, 0.3055555555555555]

In [17]:
model.predict(data[9,:-1])

1

In [7]:
chromosome = model.world.populations[0].chromosomes[2]

In [42]:
chromosome.genes[1].value = set([2])

In [43]:
chromosome.toDict()

{'uuid': '75b2c97a-2f01-45e2-b3e8-8b2fe08cfedd',
 'generation': 98,
 'genes': [{'identifier': '0', 'value': 'set()'},
  {'identifier': '1', 'value': '{2}'},
  {'identifier': '2', 'value': 'set()'},
  {'identifier': '3', 'value': '{1}'}],
 'fitness': '0.4166666666666666',
 'parents': ['b88f4ee1-3a41-49fb-979b-be609ec47e62',
  'b7a24ff9-537f-45fc-8251-1fb29fc4a906']}

In [47]:
chromosome.expression = None
~chromosome

In [48]:
chromosome.expression = chromosome.express(chromosome)

In [46]:
model.world.populations[0].fitness(chromosome.expression,2)

0.3333333333333333

In [12]:
model.world.populations[0].toDict()

{'generation': 99,
 'goal': '2',
 'chromosomes': [{'uuid': 'b7a24ff9-537f-45fc-8251-1fb29fc4a906',
   'generation': 5,
   'genes': [{'identifier': '0', 'value': 'set()'},
    {'identifier': '1', 'value': 'set()'},
    {'identifier': '2', 'value': 'set()'},
    {'identifier': '3', 'value': '{1}'}],
   'fitness': '0.4166666666666666',
   'parents': ['1d5a1ba6-f06f-4449-a35c-087290172097',
    '9529b0d5-4385-4bec-b072-4e8214e019f1']},
  {'uuid': '1485eec9-52c2-4d65-b336-74caedd5adc9',
   'generation': 99,
   'genes': [{'identifier': '0', 'value': 'set()'},
    {'identifier': '1', 'value': 'set()'},
    {'identifier': '2', 'value': 'set()'},
    {'identifier': '3', 'value': '{1}'}],
   'fitness': '0.4166666666666666',
   'parents': ['7379d1c6-bff8-4d57-bc30-64dc2e78281f',
    '3c2a611b-a9c9-4293-9ab1-11b15471a6c4']},
  {'uuid': '75b2c97a-2f01-45e2-b3e8-8b2fe08cfedd',
   'generation': 98,
   'genes': [{'identifier': '0', 'value': 'set()'},
    {'identifier': '1', 'value': 'set()'},
    {'id